### Домашнее задание к вебинару 6 

Для задание взят датасет, содержащий множество объявления из интернета.  
Признаки кодируют геометрию изображения, а также фразы, которые встречаются в URL-адресе, URL-адресе изображения и альтернативном тексте, а также слова из текста автора.

Датасет находится по ссылке: https://archive.ics.uci.edu/ml/datasets/Internet+Advertisements  
  
Задача состоит в том, чтобы определить, является ли страница рекламным объявлением или нет.

In [1032]:
# Загрузим датасет

import pandas as pd
import numpy as np
data = pd.read_csv('ad.data')
data.head(3)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,125,125.1,1.0,1,0,0.1,0.2,0.3,0.4,0.5,...,0.1532,0.1533,0.1534,0.1535,0.1536,0.1537,0.1538,0.1539,0.1540,ad.
0,57,468,8.2105,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,33,230,6.9696,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


**Описание датасета с сайта:**  
3 continous; others binary -  3 непрерывных признака и все остальные бинарные  

Непрерывные признаки получается - 125, 125.1 и 1

In [1033]:
data.shape

(3278, 1559)

Целевой признак - "ad." 

In [1034]:
data['ad.'].unique()

array(['ad.', 'nonad.'], dtype=object)

In [1035]:
#Переименуем столбец
data.rename(columns = {'ad.':'target'},inplace=True)

# Скорректируем значения классов (0- не объявление, 1 - объявление)
data['target'].replace({'nonad.': 0, 'ad.': 1}, inplace=True)

Для удобства и из-за "некрасивого" названия переименуем первые 3 непрерывные признака

In [1036]:
data.rename(columns = {' 125':'cont_feature_125', 
                      ' 125.1':'cont_feature_125.1',
                      '   1.0':'cont_feature_1.0' }, inplace = True)

In [1037]:
data.columns

Index(['cont_feature_125', 'cont_feature_125.1', 'cont_feature_1.0', '1', '0',
       '0.1', '0.2', '0.3', '0.4', '0.5',
       ...
       '0.1532', '0.1533', '0.1534', '0.1535', '0.1536', '0.1537', '0.1538',
       '0.1539', '0.1540', 'target'],
      dtype='object', length=1559)

Посмотрим на соотношение классов

In [1038]:
data['target'].value_counts()

0    2820
1     458
Name: target, dtype: int64

Выраженный дисбаланс классов

#### Проведем Data Engineering

In [1039]:
# Изучим типы данных
data.dtypes

cont_feature_125      object
cont_feature_125.1    object
cont_feature_1.0      object
1                     object
0                      int64
                       ...  
0.1537                 int64
0.1538                 int64
0.1539                 int64
0.1540                 int64
target                 int64
Length: 1559, dtype: object

**В описании датасета сказано, что в одном или более непрерывных признаков (continious features) пропущено 28% данных и эти значения неизвестны.**

In [1040]:
# Проверим, есть ли пропуски в данных
data.isnull().sum()

cont_feature_125      0
cont_feature_125.1    0
cont_feature_1.0      0
1                     0
0                     0
                     ..
0.1537                0
0.1538                0
0.1539                0
0.1540                0
target                0
Length: 1559, dtype: int64

Явных пропусков нет, однако в описании датасета было сказано, что часть значений неизвестна, значит они заполнены как-то иначе

In [1041]:
data['cont_feature_125'].unique()

array(['  57', '  33', '  60', '  59', '   ?', '  90', ' 125', '  30',
       '  35', '  72', '  50', ' 120', '   2', '  65', '  44', '  80',
       '  64', '  31', ' 100', '  74', '  93', '  21', '  20', '  55',
       ' 135', '  26', ' 136', ' 124', '  52', '  45', ' 165', '  25',
       ' 105', '  88', '  73', '  43', ' 477', '  70', '  40', ' 240',
       '  51', '  68', ' 171', '  71', '  29', '  38', '  36', '  46',
       '  12', '  95', '  41', '  96', '  16', ' 150', ' 159', ' 110',
       ' 161', '  15', ' 195', '  11', ' 114', ' 160', '  63', ' 134',
       '  24', '  62', ' 106', ' 200', '  39', ' 115', '  75', '  10',
       ' 207', '  32', '  27', '  48', ' 107', '  28', '  23', '  54',
       ' 102', ' 149', ' 119', '  76', '  79', ' 158', ' 154', '   8',
       ' 153', '  83', ' 174', ' 156', ' 128', '  14', ' 172', ' 104',
       ' 146', '  98', ' 144', ' 175', ' 116', ' 145', '  99', ' 220',
       ' 185', '  34', ' 239', ' 140', ' 262', ' 142', ' 191', '  22',
      

Видимо, знак ? вопроса и есть условно пропущенное значение

In [1042]:
len(data[data['cont_feature_125']=='   ?'])

903

In [1043]:
# Заменим эти значения на 0, вероятно, они просто неизвестны либо отсутсвуют

In [1044]:
data[data['cont_feature_125']=='   ?'] = 0

In [1045]:
# Теперь сменим тип данных 
data['cont_feature_125'] = data['cont_feature_125'].astype('int')

Проверим, встречается ли "?" или другие подозрительные значения в других признаках

In [1046]:
#Проверим сначала наши непрерывные признаки
data['cont_feature_125.1'].unique()

array([' 468', ' 230', ' 460', ' 234', 0, '  52', '  60', ' 125', ' 585',
       '  88', ' 120', ' 392', ' 107', '   2', ' 127', '  80', ' 231',
       ' 100', '  54', '  96', ' 147', '  95', ' 188', '  58', ' 126',
       ' 159', ' 183', ' 400', '  93', '  83', ' 476', '  90', '  75',
       ' 225', '  69', ' 261', ' 270', ' 142', ' 370', ' 455', ' 200',
       ' 309', '  89', ' 129', ' 470', '  43', ' 345', ' 150', ' 520',
       ' 433', ' 164', ' 215', ' 510', ' 612', ' 227', ' 440', ' 130',
       ' 430', ' 148', ' 280', ' 264', ' 416', ' 216', ' 144', ' 109',
       '  57', '  36', '  92', ' 239', ' 218', '  16', ' 209', ' 117',
       ' 111', ' 220', '   ?', ' 288', '  50', ' 140', ' 199', ' 110',
       ' 184', ' 106', '  64', ' 250', ' 116', '  35', '  46', ' 205',
       '  45', ' 101', ' 600', '  32', '  41', ' 178', ' 260', '  51',
       '  40', '  74', '  26', '  30', ' 119', ' 299', ' 179', ' 182',
       ' 162', ' 108', '  82', ' 124', ' 186', ' 152', ' 242', ' 139',
   

In [1047]:
data[data['cont_feature_125.1']=='   ?'] = 0
# также переведем в целочисленный тип данных
data['cont_feature_125.1'] = data['cont_feature_125.1'].astype('int')

In [1048]:
data['cont_feature_1.0'].unique()

array(['8.2105', '6.9696', '   7.8', '7.7966', '   3.9', 0, '0.5777',
       '0.6666', '   1.0', '  19.5', '1.4666', '   2.0', '3.5714',
       '5.4444', '  2.14', '1.0416', '1.9230', '2.8863', '3.6093',
       '2.8387', '0.7297', '3.0967', '1.5806', '1.5833', '8.9523',
       '   2.9', '8.3636', '0.9333', '   5.3', '8.7142', '   8.0',
       '  18.0', '0.6838', '0.9677', '  4.15', '9.1538', '2.2222',
       '0.5454', '   3.0', '2.1428', '6.6666', '0.9452', '1.3333',
       '2.8064', '8.1818', '4.3030', '5.3488', '   7.4', '   9.1',
       '  6.18', '1.4833', '0.2704', '6.7142', '0.7166', '  10.0',
       '7.6666', '2.9032', '   2.5', '5.9090', '   0.5', '8.4901',
       '2.7796', '2.3888', '   5.1', '   9.0', '1.3274', '7.3333',
       '2.1666', '7.1666', '2.3098', '5.1034', '4.6666', '6.9473',
       '7.5636', '   6.0', '2.3695', '  4.75', '1.5789', '4.3333',
       '1.0285', '  1.15', '   4.5', '5.8292', '2.2708', '   1.5',
       '1.3144', '  9.75', '1.0090', '1.3664', '1.4769', '8

In [1049]:
data['cont_feature_1.0'] = data['cont_feature_1.0'].astype('float')

In [1050]:
# Проверим также признак '1', который имеет тип object 

data['1'].unique()

array(['1', 0, '0', '?', 1], dtype=object)

In [1051]:
data[data['1'] == '1'] = 1
data[data['1'] == '0'] = 0

In [1052]:
data['1'].unique()

array([1, 0, '?'], dtype=object)

In [1053]:
# как и предполагалось, признак содержит неизвестные значения '?', оценим их количество
len(data[data['1']== '?'])

10

In [1054]:
data['1'].value_counts()

1    1791
0    1477
?      10
Name: 1, dtype: int64

In [1055]:
data[data['1'] == '?'] = 0

In [1056]:
data['1'] = data['1'].astype('int')

In [1057]:
data.dtypes

cont_feature_125        int64
cont_feature_125.1      int64
cont_feature_1.0      float64
1                       int64
0                       int64
                       ...   
0.1537                  int64
0.1538                  int64
0.1539                  int64
0.1540                  int64
target                  int64
Length: 1559, dtype: object

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [1058]:
from sklearn.model_selection import train_test_split

X_data = data.drop('target', axis = 1)

#Скорректировала значения таргета, иначе результат был слишком идеальный (по 100%)

data['target'] == 0
pos_ix = np.random.randint(0, data.shape[0], size = 458)
data['target'][pos_ix] = 1

y_data = data['target']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle = True, random_state=7)

<ipython-input-1058-fbf9810fce58>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'][pos_ix] = 1


**Обучим классификатор**

In [1059]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train, y_train)
y_predict = model.predict(X_test)

Проверяем качество

In [1060]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    #print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    #print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    #print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    #print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    #print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)
print(f1, roc, rec, prc)

0.8790786948176583 0.8928066429418743 0.8149466192170819 0.9541666666666667


### Переходим к Positive Unlabeled learning

Представим, что нам неизвестны негативы и часть позитивов

In [1061]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
# оставляем только 25% всех позитивных объектов
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 347/1386 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [1062]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    2931
 1     347
Name: class_test, dtype: int64


* Теперь мы имеет 293 позитивно размеченных объектов (метка 1) в столбце class_test, а остальные отмечены как -1
* Колонка target при этом содержит истинное распределение классов

In [1063]:
mod_data.head(10)

,cont_feature_125,cont_feature_125.1,cont_feature_1.0,1,0,0.1,0.2,0.3,0.4,0.5,...,0.1533,0.1534,0.1535,0.1536,0.1537,0.1538,0.1539,0.1540,target,class_test
0,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
2,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
4,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
5,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
6,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
7,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1,1,1.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1
9,0,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1


Нельзя забывать, что этот датафрейм содержит бывшую целевую переменную target, которую мы сохраняем здесь только для сравнения результатов  
  
[:-2] - это истинные метки классов для позитивных и негативных классов, [:-1] - новые метки для позитивного и неразмеченного классов

In [1064]:
x_data = mod_data.iloc[:,:-2].values # X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

#### Применим random negative sampling

In [1065]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(347, 1560) (347, 1560)


In [1066]:
model = RandomForestClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
f1_1, roc_1, rec_1, prc_1 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
print(f1, roc, rec, prc)

0.8790786948176583 0.8928066429418743 0.8149466192170819 0.9541666666666667


**6. Сравним метрики**

In [1067]:
comparing_models = pd.DataFrame({'classifier': ['Positive-Negative','Positive-Unlabeled 0.25'],
                                 'f1 score': [f1, f1_1],
                                 'roc_auc': [roc, roc_1],
                                 'recall': [rec, rec_1],
                                 'precision': [prc, prc_1]})
comparing_models

,classifier,f1 score,roc_auc,recall,precision
0,Positive-Negative,0.879079,0.892807,0.814947,0.954167
1,Positive-Unlabeled 0.25,0.831650,0.866359,0.805435,0.859629


Результат на классическом классификаторе Positive Negative получился выше

**7. Поэкспериментируем с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)**

In [1068]:
P_board = [0.35, 0.5, 0.7]
for board in P_board:
    mod_data = data.copy()

    pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
    np.random.shuffle(pos_ind)

    pos_sample_len = int(np.ceil(board * len(pos_ind)))
    print(f'Using part {board} we will have {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
    pos_sample = pos_ind[:pos_sample_len]
    
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    #print('target variable:\n', mod_data.iloc[:,-1].value_counts())
    
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    print(f'Board: {board}, Neg_sample sample: {neg_sample.shape}, Pos_sample sample: {pos_sample.shape}\n\n')
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

    model = RandomForestClassifier()

    model.fit(sample_train.iloc[:,:-2].values, 
              sample_train.iloc[:,-2].values)
    y_predict = model.predict(sample_test.iloc[:,:-2].values)
    f1, roc, rec, prc = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
    
    comparing_models.loc[comparing_models.shape[0]+1] = [f'Positive-Unlabeled {board}', f1, roc, rec, prc]
    
print(comparing_models)

Using part 0.35 we will have 486/1386 as positives and unlabeling the rest
Board: 0.35, Neg_sample sample: (486, 1560), Pos_sample sample: (486, 1560)


Using part 0.5 we will have 693/1386 as positives and unlabeling the rest
Board: 0.5, Neg_sample sample: (693, 1560), Pos_sample sample: (693, 1560)


Using part 0.7 we will have 971/1386 as positives and unlabeling the rest
Board: 0.7, Neg_sample sample: (971, 1560), Pos_sample sample: (971, 1560)


                classifier  f1 score   roc_auc    recall  precision
0        Positive-Negative  0.879079  0.892807  0.814947   0.954167
1  Positive-Unlabeled 0.25  0.831650  0.866359  0.805435   0.859629
3  Positive-Unlabeled 0.35  0.844875  0.880642  0.817694   0.873926
4   Positive-Unlabeled 0.5  0.805611  0.864766  0.794466   0.817073
5   Positive-Unlabeled 0.7  0.801653  0.870699  0.779116   0.825532


Вывод: Можно ответить увеличение площади под ROC кривой при увеличении доли позитивных объектов в модели, однако  recall, recall и f1 score снижаются.  
В целом, в данной задаче при варьировании доли положительных объектов, существенного варьирования метрик не наблюдается. 

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Я предполагаю, что метод 2 step approach будет являться предпочтительнее, т. к. он представляет собой усовершенствованный метод random negative sampling. Основываясь на reliable negative можно построить более точную модель.